In [24]:
from datasets import load_dataset

dataset = load_dataset('tasksource/bigbench', 'cs_algorithms', split='train')

Found cached dataset bigbench (/Users/sunil/.cache/huggingface/datasets/tasksource___bigbench/cs_algorithms/1.0.0/d9f074b582e19cb8b59cbcd48e43c1c15cf316f7c774c526da2bca369f786cc3)


#### Test Prompt Templates

In [25]:
# Following self-instruct prompt template (appendix of self-instruct paper):
import numpy as np
k_idx = np.random.randint(low=0, high=len(dataset), size=3).tolist()
k = 4

small_dataset = dataset.select(k_idx)

new = "\n"
examples = []
for i in range(len(small_dataset)):
  data = small_dataset[i]
  
  options = data['multiple_choice_targets']
  options = [f"Option {j}: {o}{new}" for j,o in enumerate(options)]
  options = "".join(options)
  
  targets = data['targets']
  targets = "\n".join(targets)

  example = f"Question {i}:{new}Input: {data['inputs']}{new}Options:{new}{options}Target(s):{new}{targets}"
  examples.append(example)
  print(example)
  print('-'*50)

Question 0:
Input: Given two strings, determine the length of the longest common subsequence.

Strings: ASZKZUPCKE ZJISW
Length of longest common subsequence:
Options:
Option 0: 0
Option 1: 1
Option 2: 2
Option 3: 3
Option 4: 4
Option 5: 5
Option 6: 6
Option 7: 7
Option 8: 8
Option 9: 9
Target(s):
1
--------------------------------------------------
Question 1:
Input: Determine whether the given sequence of parentheses is properly matched.

Sequence: [ [ { } ( ) { } ( ) ] ] ( ) ( [ ] { } )
Valid/Invalid?
Options:
Option 0: Valid
Option 1: Invalid
Target(s):
Valid
--------------------------------------------------
Question 2:
Input: Determine whether the given sequence of parentheses is properly matched.

Sequence: [ ) { ]
Valid/Invalid?
Options:
Option 0: Valid
Option 1: Invalid
Target(s):
Invalid
--------------------------------------------------


In [26]:
small_dataset.to_json("test.json")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 24.76ba/s]


792

In [30]:
import json
import pprint

n_questions = 10
keys = ['inputs', 'targets', 'multiple_choice_targets']

ds_dict = small_dataset.to_dict() # each key has same amount of values want to map each value of each key with each other
n_examples = len(ds_dict['inputs'])

examples = []
for i in range(n_examples):
    example = {'inputs':"", 'targets':"", 'multiple_choice_targets':""}
    for key in keys:
      example[key] = ds_dict[key][i]
    examples.append(example)

print(len(examples))
prompt = f"Generate a series of {n_questions} questions in a valid JSON format:"
for i, e in enumerate(examples):
   prompt += f"\nQuestion {i}:\n{e}"

prompt += f"\nQuestion {n_examples}:"

# create string prompt
pprint.pprint(prompt)


3
('Generate a series of 10 questions in a valid JSON format:\n'
 'Question 0:\n'
 "{'inputs': 'Given two strings, determine the length of the longest common "
 'subsequence.\\n\\nStrings: ASZKZUPCKE ZJISW\\nLength of longest common '
 "subsequence:', 'targets': ['1'], 'multiple_choice_targets': ['0', '1', '2', "
 "'3', '4', '5', '6', '7', '8', '9']}\n"
 'Question 1:\n'
 "{'inputs': 'Determine whether the given sequence of parentheses is properly "
 'matched.\\n\\nSequence: [ [ { } ( ) { } ( ) ] ] ( ) ( [ ] { } '
 ")\\nValid/Invalid?', 'targets': ['Valid'], 'multiple_choice_targets': "
 "['Valid', 'Invalid']}\n"
 'Question 2:\n'
 "{'inputs': 'Determine whether the given sequence of parentheses is properly "
 "matched.\\n\\nSequence: [ ) { ]\\nValid/Invalid?', 'targets': ['Invalid'], "
 "'multiple_choice_targets': ['Valid', 'Invalid']}\n"
 'Question 3:')


In [14]:
ds_dict.keys()

dict_keys(['inputs', 'targets', 'multiple_choice_targets', 'multiple_choice_scores', 'idx'])

### Input / Output Chat Responses

In [33]:
import openai
import json
from private import sk

openai.api_key = sk
completion = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[{"role": "system", "content": prompt}],  # change role??
                temperature=0,
            )

with open('icl_generation_example_response.json', 'w') as f:
    json.dump(completion, f)

In [41]:
with open('icl_generation_example_response.json', 'r') as f:
  data = json.load(f)

msg_content = data['choices'][0]['message']['content']
generate_qs = msg_content.split("Question")
f"In-context questions: {len(examples)}; generated questions: {len(generate_qs)}"

# print(data['choices'][0]['message']['content'])

'In-context questions: 3; generate questions: 7'

In [78]:
import ast 

generated_json_examples = []
for i, q in enumerate(generate_qs):
    if i == 0: # end of prompt = "... Question i:"
        msg_dict = ast.literal_eval(q)
    else: # cut off Question:\n that comes before the {<json_question>}
        q = q[q.find('\n'):]
        q = q.rstrip("\n")
        msg_dict = ast.literal_eval(q)
    generated_json_examples.append(msg_dict)

with open("generated_questions.json", "a") as f:
    json.dump(generated_json_examples, f)

In [65]:
print(generate_qs[2])

 5:
{'inputs': 'Given a list of integers, find the second largest number.\n\nList: [1, 5, 3, 2, 4]\nSecond largest number:', 'targets': ['4'], 'multiple_choice_targets': ['1', '2', '3', '4', '5']}



In [66]:
msg_content

"{'inputs': 'Given a list of integers, find the maximum sum of any contiguous subarray.\\n\\nList: [1, -2, 3, 4, -5, 8]\\nMaximum sum:', 'targets': ['10'], 'multiple_choice_targets': ['-2', '3', '10', '12', '13']}\nQuestion 4:\n{'inputs': 'Given a string, determine whether it is a palindrome.\\n\\nString: racecar\\nPalindrome/Not Palindrome?', 'targets': ['Palindrome'], 'multiple_choice_targets': ['Palindrome', 'Not Palindrome']}\nQuestion 5:\n{'inputs': 'Given a list of integers, find the second largest number.\\n\\nList: [1, 5, 3, 2, 4]\\nSecond largest number:', 'targets': ['4'], 'multiple_choice_targets': ['1', '2', '3', '4', '5']}\nQuestion 6:\n{'inputs': 'Given a list of integers, find the sum of all even numbers.\\n\\nList: [1, 2, 3, 4, 5, 6, 7, 8, 9]\\nSum of even numbers:', 'targets': ['20'], 'multiple_choice_targets': ['10', '12', '14', '16', '18', '20']}\nQuestion 7:\n{'inputs': 'Given a list of integers, find the product of all odd numbers.\\n\\nList: [1, 2, 3, 4, 5, 6, 7, 